## Portfolio Simulator Using Data Robot Prediction Output

In [269]:
#Libraries
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np

#Initialization Variables
initial_portfolio_size = 100000000
per_share_trading_cost = 0.05
number_of_holdings = 10
minimum_cash_percent = 0.005
sell_percentile = 0.30 #sell above X percentile
min_lot_size = 100
num_current_holdings = 0
num_to_sell = 0
num_to_buy = 0


### Read in data and compute universe and classifier returns and size

In [270]:
#read data and get unique rebalance dates
#df_raw_predictions = pd.read_csv(r'G:\Python Data\DataRobot Exports\1B Retirement - Holdout with Returns.csv')
df_raw_predictions = pd.read_csv(r'G:\Data Robot\Retirement\1B Retirement - Holdout with Returns.csv')
df_rebalance_dates = pd.DataFrame({'Rebalance Date': pd.unique(df_raw_predictions['Period (YYYYMMDD)'])})


#make copy of raw master data
df_ranked_data = df_raw_predictions.drop(['timestamp','Partition','Identifier','row_id'], axis=1)

#rank and percentile the prediction scores, outperformers (1) and underperformers (0)
df_ranked_data['Rank'] = df_ranked_data.groupby('Period (YYYYMMDD)')['Training Prediction'].rank(ascending = False)
df_ranked_data['Percentile'] = df_ranked_data.groupby('Period (YYYYMMDD)')['Training Prediction'].rank(ascending = False, pct=True)
df_outperformers = df_ranked_data[df_ranked_data['PredictedLabel'] == 1]
df_underperformers = df_ranked_data[df_ranked_data['PredictedLabel'] == 0]

#calculate returns
#universe
sr_universe_returns = df_ranked_data.groupby('Period (YYYYMMDD)')['Universe Returns'].mean()
sr_universe_size = df_ranked_data.groupby('Period (YYYYMMDD)')['Universe Returns'].size()
sr_outperformers_returns = df_outperformers.groupby('Period (YYYYMMDD)')['Universe Returns'].mean()
sr_outperformers_size = df_outperformers.groupby('Period (YYYYMMDD)')['Universe Returns'].size()
sr_underperformers_returns = df_underperformers.groupby('Period (YYYYMMDD)')['Universe Returns'].mean()
sr_underperformers_size = df_underperformers.groupby('Period (YYYYMMDD)')['Universe Returns'].size()

#convert to a dataframe
df_universe_returns = sr_universe_returns.to_frame()
df_universe_size = sr_universe_size.to_frame()
df_outperformers_returns = sr_outperformers_returns.to_frame()
df_outperformers_size = sr_outperformers_size.to_frame()
df_underperformers_returns = sr_underperformers_returns.to_frame()
df_underperformers_size = sr_underperformers_size.to_frame()

#rename columns
df_universe_returns.rename(columns={'Universe Returns':'Universe Return'}, inplace=True)
df_universe_size.rename(columns={'Universe Returns':'Universe Size'}, inplace=True)
df_outperformers_returns.rename(columns={'Universe Returns':'Outperformers Return'}, inplace=True)
df_outperformers_size.rename(columns={'Universe Returns':'Outperformers Size'}, inplace=True)
df_underperformers_returns.rename(columns={'Universe Returns':'Underperformers Return'}, inplace=True)
df_underperformers_size.rename(columns={'Universe Returns':'Underperformers Size'}, inplace=True)



### Portfolio Simulator

In [271]:
#Reset the starting portfolio
df_portfolio = df_ranked_data[0:0]
df_portfolio['Add Date'] = 0
df_previous_portfolio = df_ranked_data[0:0]
df_previous_portfolio['Add Date'] = 0
num_current_holdings = len(df_previous_portfolio)

#Now Run Through Loop of Every Period - This only sells if company is no longer considered an outperformer
#Sells are then the next highest rated candidates not already owned

for i in range(df_rebalance_dates.size):
    
    #this is the full universe for the current period
    df_current_universe = df_ranked_data[df_ranked_data['Period (YYYYMMDD)'] == int(df_rebalance_dates.iloc[i:i+1].values[0])]
    
    #go through buy candidates and add to holdings
    df_buy_candidates = df_ranked_data[(df_ranked_data['Period (YYYYMMDD)'] == int(df_rebalance_dates.iloc[i:i+1].values[0])) & (df_ranked_data['PredictedLabel'] == 1)]
    df_buy_candidates = df_buy_candidates.sort_values(by='Rank')
    df_buy_candidates.reset_index(drop=True, inplace=True)

    #transfer new ranks to previous portfolio
    df_new_portfolio = df_current_universe[df_current_universe.Ticker.isin(df_previous_portfolio.Ticker)]
    df_new_portfolio = df_new_portfolio[df_new_portfolio['PredictedLabel'] == 1]
    
    #add back the add dates
    df_previous_add_dates = pd.DataFrame(columns=['Ticker', 'Add Date'])
    df_previous_add_dates['Ticker'] = df_previous_portfolio['Ticker']
    df_previous_add_dates['Add Date'] = df_previous_portfolio['Add Date']    
    df_new_portfolio = pd.merge(df_new_portfolio, df_previous_add_dates, on='Ticker')
    df_new_portfolio.reset_index(inplace=True)

    #the final buys are the ones not already owned
    df_real_buys = df_buy_candidates[(~df_buy_candidates.Ticker.isin(df_new_portfolio.Ticker))]
    df_real_buys.reset_index(inplace=True)    
        
    num_to_buy = number_of_holdings - len(df_new_portfolio)

    df_new_buys = df_real_buys[0:num_to_buy]
    df_new_buys['Add Date'] = int(df_rebalance_dates.iloc[i:i+1].values[0])
    df_new_portfolio = df_new_portfolio.append(df_new_buys, ignore_index=True, sort=False)
    df_new_portfolio.drop(columns='index',inplace=True)
    df_new_portfolio['Period (YYYYMMDD)'] = int(df_rebalance_dates.iloc[i:i+1].values[0])
    df_new_portfolio = df_new_portfolio.sort_values(by='Add Date')
    
    #append to master portfolio
    df_portfolio = df_portfolio.append(df_new_portfolio, ignore_index=True, sort=False)
    
    #last step - new portfolio is not the previous portfolio
    df_previous_portfolio = df_new_portfolio
    
#Calculate Holdings Returns
sr_portfolio_returns = df_portfolio.groupby('Period (YYYYMMDD)')['Universe Returns'].mean()
sr_portfolio_size = df_portfolio.groupby('Period (YYYYMMDD)')['Universe Returns'].size()
df_portfolio_returns = sr_portfolio_returns.to_frame()
df_portfolio_size = sr_portfolio_size.to_frame()
df_portfolio_returns.rename(columns={'Universe Returns':'Portfolio Return'}, inplace=True)
df_portfolio_size.rename(columns={'Universe Returns':'Portfolio Size'}, inplace=True)
    
#save holdings
df_portfolio.to_csv(r'G:\Data Robot\Retirement\Holdings3.csv',index=False)

C:\Users\jnguyen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Insert Portfolio Results into Results Table

In [272]:
#merged_table
df_results_table = pd.merge(df_outperformers_returns, df_underperformers_returns, on='Period (YYYYMMDD)')
df_results_table = pd.merge(df_results_table, df_universe_returns, on='Period (YYYYMMDD)')
df_results_table = pd.merge(df_results_table, df_portfolio_returns, on='Period (YYYYMMDD)')
df_results_table = pd.merge(df_results_table, df_outperformers_size, on='Period (YYYYMMDD)')
df_results_table = pd.merge(df_results_table, df_underperformers_size, on='Period (YYYYMMDD)')
df_results_table = pd.merge(df_results_table, df_universe_size, on='Period (YYYYMMDD)')
df_results_table = pd.merge(df_results_table, df_portfolio_size, on='Period (YYYYMMDD)')

#df_results_table.to_csv(r'G:\Python Data\FactSet Alpha Testing Export\Results Table.csv',index=True)
df_results_table.to_csv(r'G:\Data Robot\Retirement\Results Table.csv',index=True)

In [267]:
number_of_holdings

10

In [204]:
len(df_real_buys)



20

In [209]:
len(df_merged)

23